1. **Restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart)
2. **Run all cells** (in the menubar, select Cell$\rightarrow$Run All).
3. __Use the__ `Validate` __button in the Assignments tab before submitting__.

__Include comments, derivations, explanations, graphs, etc.__ 

You __work in groups__ (= 3 people). __Write the full name and S/U-number of all team members!__

---

# Assignment 4 (Statistical Machine Learning 2024)
# **Deadline: 21 December 2024**

## Instructions
* Fill in any place that says `YOUR CODE HERE` or `YOUR ANSWER HERE` __including comments, derivations, explanations, graphs, etc.__ 
Elements and/or intermediate steps required to derive the answer have to be in the report. If an exercise requires coding, explain briefly what the code does (in comments). All figures should have titles (descriptions), axis labels, and legends.
* __Please use LaTeX to write down equations/derivations/other math__! How to do that in Markdown cells can be found [here](https://www.fabriziomusacchio.com/blog/2021-08-10-How_to_use_LaTeX_in_Markdown/), a starting point for various symbols is [here](https://www.overleaf.com/learn/latex/Mathematical_expressions).
* Please do __not add new cells__ to the notebook, try to write the answers only in the provided cells. Before you turn the assignment in, make sure everything runs as expected.
* __Use the variable names given in the exercises__, do not assign your own variable names. 
* __Only one team member needs to upload the solutions__. This can be done under the Assignments tab, where you fetched the assignments, and where you can also validate your submissions. Please do not change the filenames of the individual Jupyter notebooks.

For any problems or questions regarding the assignments, ask during the tutorial or send an email to charlotte.cambiervannooten@ru.nl and janneke.verbeek@ru.nl .

## Introduction
Assignment 4 consists of:
1. Bayesian inference in binary response problem (50 points);
2. The EM algorithm for doping detection (50 points)__;
3. __Gibbs sampling and Metropolis-Hastings (50 points)__;
4. State-Space models (50 points).

## Libraries

Please __avoid installing new packages__, unless really necessary.

In [ ]:
import IPython
assert IPython.version_info[0] >= 3, "Your version of IPython is too old, please update it to at least version 3."

import random
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal

# Set fixed random seed for reproducibility
np.random.seed(2022)

## Gibbs sampling and Metropolis-Hastings (50 points)
Exact inference is often not tractable in real-world probabilistic models. By *exact*, we mean that the likelihood and posterior are available in closed form (as an analytical expression). For example, recall *Gaussian processes regression*: we have a Gaussian process prior and a Gaussian likelihood that conveniently result in a Gaussian process posterior. It is, however, not the case when we want to use Gaussian processes for classification. We have a Gaussian process prior, yet, the likelihood has to be, for example, sigmoid (logistic) or cumulative normal (probit). Such a prior and likelihood will not give us the Gaussian process posterior anymore. Thus, some other methods would have to be used for this purpose, such as *Laplace approximation*, *Expectation Propagation*, or *Markov Chain Monte Carlo methods*. In particular, for the Gaussian processes classification problem, one could use **Gibbs sampling**. In this exercise we will avoid some cumbersome derivations related to more complex problems (such as GP classification) and implement Gibbs sampling and Metropolis-Hastings for toy problems to illustrate the intuition behind the algorithms. 
### Gibbs sampler
Consider a two-dimensional Gaussian distribution $\boldsymbol{x}\sim N(\boldsymbol{\mu}, \boldsymbol{\Sigma})$ with parameters 
\begin{equation}
\boldsymbol{\mu}=
\begin{bmatrix}
1\\
2
\end{bmatrix}\quad
\boldsymbol{\Sigma}=\begin{bmatrix}
1&0.8\\
0.8&2
\end{bmatrix}
\end{equation}
1. Write down the relevant conditional densities for sampling from a 2D Gaussian, write down the iterations of the Gibbs sampler for sampling from a 2D Gaussian. 

YOUR ANSWER HERE

2. Implement the Gibbs sampler. Start with an initial guess $[-1.5, 4]^{T}$. Plot the Gaussian contours along with the initial guess after 2, 5, and 100 full cycles of the Gibbs sampler. 

First, create a function that plots the contours, the initial guess, and the generated samples. 

In [ ]:
def gauss_contour(mu, sigma, init_guess, samples=[], title='Initial guess'):
    """
    Function to plot Gaussian contours and initial guess.

    Parameters
    ----------
    mu : (2,) np.ndarray
        The mean.
    sigma : (2,2) np.ndarray
        The covariance.
    init_guess : (2,) np.ndarray
        The array containing the initial guess.
    samples : (T,2) np.ndarray
        The array containing the generated samples.
    title : The title of the plot.

    Returns
    -------
    """
    # YOUR CODE HERE
    raise NotImplementedError()

Implement the Gibbs sampler.

In [ ]:
def Gibbs(mu, sigma, init_guess, T):
    """
    Function to plot Gaussian contours and initial guess.

    Parameters
    ----------
    mu : (2,) np.ndarray
        The mean.
    sigma : (2,2) np.ndarray
        The covariance.
    init_guess : (2,) np.ndarray
        The array containing the initial guess.
    T : integer
        The number of samples to generate.

    Returns
    -------
    The samples generated.
    """
    # YOUR CODE HERE
    raise NotImplementedError()

Run the Gibbs sampler and plot the results.

In [ ]:
"""
Define the initial guess, the mean, the covariance, and run the Gibbs sampler.
"""
# YOUR CODE HERE
raise NotImplementedError()

3. Compute the sample mean and sample covariance after 50, 100, 500, and 1000 samples.

In [ ]:
"""
Compute the sample mean and covariance.
"""
# YOUR CODE HERE
raise NotImplementedError()

Comment on the results.

YOUR ANSWER HERE

### Metropolis-Hastings
Here, we will continue working with bivariate Gaussian distribution. However, we will set up the problem differently. Assume now that the correlation parameter $\rho$ is unknown (while the mean and variances remain known):
\begin{equation}
\boldsymbol{\mu}=
\begin{bmatrix}
0\\
0
\end{bmatrix}\,\,\,\,\,\,\,\,\,\,
\boldsymbol{\Sigma}=\begin{bmatrix}
1&\rho\\
\rho&1
\end{bmatrix}
\end{equation}
The likelihood function follows
\begin{equation}
p(x_{1}^{1:N}, x_{2}^{1:N}|\rho)=\prod_{i=1}^{N}\frac{1}{2\pi\sqrt{1-\rho^{2}}}\exp\left(-\frac{1}{2(1-\rho^{2})}[(x_{1}^{i})^{2}-2\rho x_{1}^{i}x_{2}^{i}+(x_{2}^{i})^{2}] \right),
\label{eq:likelihood}
\tag{1}
\end{equation}
where index $i$ represents the $i^{th}$ observation. Further, we want to specify an uninformative prior on our covariance matrix. It is common to take the Jeffreys prior (https://en.wikipedia.org/wiki/Jeffreys_prior). In our case, it will become
\begin{equation}
p(\rho)=\frac{1}{|\Sigma^{3/2}|}=\frac{1}{\begin{vmatrix} 1&\rho\\
	\rho&1\end{vmatrix}^{3/2}}=1/(1-\rho^{2})^{3/2}.
\label{eq:jeffrey}
\tag{2}
\end{equation}
If you use Bayes's rule for finding the posterior of $p(\rho)$ you will realize that it is not of any known form. So further we will infer $p(\rho|x_{1}^{1:N}, x_{2}^{1:N})$ using the Metropolis-Hastings algorithm.
For this exercise, generate $N=1000$ data points from multivariate normal with the parameters:
\begin{equation}
\boldsymbol{\mu}=
\begin{bmatrix}
0\\
0
\end{bmatrix}\,\,\,\,\,\,\,\,\,\,
\boldsymbol{\Sigma}=\begin{bmatrix}
1&0.4\\
0.4&1
\end{bmatrix},
\end{equation}
so we know the ground thruth $\rho=0.4$. 
1. In this exercise we will implement the Metropolis-Hastings algorithm. Let us first specify the proposal distribution:
\begin{equation}
\rho^{cand}\sim Uniform(\rho^{k-1}-0.07, \rho^{k-1}+ 0.07), 
\end{equation}
where $k$ is the interation of the Metropolis-Hastings algorithm. Comment on this proposal distribution (is it symmetric? does that help?). Write down the formula for the Metropolis-Hastings acceptance function using this proposal distribution. Comment on the role of acceptance function in this algorithm.

YOUR ANSWER HERE

2. Implement the Metropolis-Hastings algorithm for this problem and the proposal function defined above.

    **Note:** We added the functions `log_joint_probability` and `acceptance_function` to help you with the implementation, but their use is optional.

In [ ]:
def log_joint_probability(X, rho):
    """
    Calculates log of joint probability.

    Parameters
    ----------
    X : (N, 2) np.ndarray
        Data points.
    rho : float
        Correlation parameter.

    Returns
    -------
    log-probability of X and rho
    """
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
def acceptance_function(X, rho_candidate, rho_current):
    """
    Computes acceptance probability of a candidate.

    Parameters
    ----------
    X : (N, 2) np.ndarray
        Data points.
    rho_candidate : float
        Candidate sample.
    rho_current : float
        Current sample.

    Returns
    -------
    Value of acceptance function for given rho_candidate and rho_current.
    """
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
def metropolis_hastings(X, half_interval_len, T):
    """
    Metropolis-Hastings sampling implementation and plotting of the results.

    Parameters
    ----------
    X : (N, 2) np.ndarray
        Data points.
    half_interval_len : float
        Half interval length for the proposal uniform distribution centered around the current state.
    T : integer
        Number of iterations.

    Returns
    -------
    Accepted samples.
    """
    # YOUR CODE HERE
    raise NotImplementedError()

Run Metropolis-Hastings for 10000 iterations. You can use the initial value of $\rho^{0}=0$. After sampling $\rho$ 10000 times report the acceptance rate, the mean, and the standard deviation of the samples. Plot the trace plot for $\rho$ (i.e., value $\rho^{k}$ at each iteration of the algorithm). Plot the histogram of the posterior distribution of $\rho$ based on the samples in the chain.

In [ ]:
"""
Run Metropolis-Hastings and report the acceptance rate, the mean and the standard deviation.
Plot the trace plot and the histogram.
"""
# YOUR CODE HERE
raise NotImplementedError()

3. Try changing the proposal distribution to 
\begin{equation}
\rho^{cand}\sim Uniform(\rho^{k-1}-0.01, \rho^{k-1}+ 0.01), 
\end{equation}
and 
\begin{equation}
\rho^{cand}\sim Uniform(\rho^{k-1}-0.3, \rho^{k-1}+ 0.3).
\end{equation}
Run Metropolis-Hastings with the new proposal distributions. Once again, report the acceptance rate, the mean and the standard deviation. Plot the trace plot and the histogram.

In [ ]:
"""
Run Metropolis-Hastings with the new proposal distributions.
Once again, report the acceptance rate, the mean and the standard deviation. Plot the trace plot and the histogram.
"""
# YOUR CODE HERE
raise NotImplementedError()

Comment on how changing the proposal distribution changes your results.

YOUR ANSWER HERE